# 07.4 Pricing & Hedging in a Jump–Diffusion Model

<h3><span style="color:#800000;"><strong>Authored by:</strong> <em>Alexandre Mathias DONNAT, Sr</em></span></h3>

**Goal of this notebook**:
- Moves from martingale conditions to risk-neutral dynamics,
- Derives option pricing formulas under jumps,
- Implements the Merton mixture series,
- Illustrates why the market becomes incomplete,
- Computes and simulates the optimal quadratic-risk hedge,
- Compares hedging errors with and without jump

**This notebook**: 

Develop the pricing framework for jump–diffusion models, derive the risk-neutral dynamics, compute European option prices (Merton mixture formula), and study quadratic-risk–minimizing hedging strategies in incomplete markets.

# 1. Risk-Neutral Measure

We consider the jump–diffusion:

$$dX_t = \mu X_{t-} dt + \sigma X_{t-} dW_t + X_{t-} dZ_t, \quad Z_t = \sum_{j=1}^{N_t} U_j,$$

with Poisson jumps of intensity $\lambda$ and i.i.d. amplitudes $U_j$.

We change measure only on the Brownian part:

$$\frac{dP^*}{dP} = \exp\left(\theta W_T - \frac{1}{2}\theta^2 T\right), \quad \theta = \frac{r - \mu - \lambda \mathbb{E}[U_1]}{\sigma}.$$

Under $P^*$:

$$W_t^* = W_t - \theta t, \quad N_t, \, U_j \text{ unchanged}.$$

The discounted asset $\tilde{X}_t = e^{-rt} X_t$ is a martingale.

Thus under $P^*$:

$$dX_t = X_{t-} \left[(r - \lambda \mathbb{E}[U_1]) dt + \sigma dW_t^* + dZ_t\right].$$


# 2. Market Incompleteness

Jumps cannot be hedged with continuous trading.
Therefore, infinitely many risk-neutral measures exist (any adjustment on the jump compensator is admissible).

We select the simplest one (Brownian-only Girsanov) to define:

$$V_t = \mathbb{E}^*\left[e^{-r(T-t)} h(X_T) \mid \mathcal{F}_t\right].$$

This is a pricing measure but hedging is no longer perfect.


# 3. European Option Pricing — Merton Mixture Formula

For payoff $h(X_T) = f(X_T)$,

$$F(t,x) = \mathbb{E}^*\left[e^{-r(T-t)} f(X_T^{t,x})\right].$$

Under $P^*$,

$$X_T^{t,x} = x \, e^{(r-\lambda m - \sigma^2/2)(T-t) + \sigma(W_T - W_t)} \prod_{j=1}^{N_{T-t}} (1+U_j), \quad m = \mathbb{E}[U_1].$$

Condition on the number of jumps $N_{T-t} = n$.

Let:

$$x_n = x \, e^{-\lambda(T-t)m} \prod_{j=1}^{n} (1+U_j).$$

Then

$$F(t,x) = \sum_{n=0}^{\infty} \mathbb{E}[F_0(t, x_n)] \, e^{-\lambda(T-t)} \frac{(\lambda(T-t))^n}{n!},$$

where $F_0$ is the Black–Scholes price with spot $x_n$ and volatility $\sigma$.

**Practical computation:** sum $n = 0, \ldots, 20$, rarely more


# 4. Numerical Implementation : Merton Call Price


We implement the full Merton mixture formula for lognormal jumps, where

$(1+U_j) = e^{Y_j}$ with $Y_j \sim N(\mu_J, \sigma_J^2)$.

Conditioned on $N_T = n$, the terminal log-price is:

$$\log X_T = \log x + \left(r - \lambda k - \frac{1}{2}\sigma^2\right)\tau + \sigma W_\tau + n\mu_J + \frac{1}{2}n\sigma_J^2 + \sigma_J \sum_{j=1}^n Z_j,$$

with jump compensator

$$k = \mathbb{E}[e^Y - 1] = e^{\mu_J + \frac{1}{2}\sigma_J^2} - 1.$$

Thus the Black–Scholes effective spot after $n$ jumps is:

$$x_n = x \exp\left(n\mu_J + \frac{1}{2}n\sigma_J^2 - \lambda k\tau\right)$$

and the call price is

$$C = \sum_{n=0}^{n_{\max}} w_n \, C_{BS}(x_n), \quad w_n = e^{-\lambda\tau} \frac{(\lambda\tau)^n}{n!}.$$


In [7]:
import numpy as np
import math
from scipy.stats import norm

In [14]:
# Black–Scholes call
def bs_call(x, K, r, sigma, tau):
    if x <= 0:
        return 0.0
    d1 = (np.log(x/K) + (r + 0.5*sigma**2)*tau) / (sigma*np.sqrt(tau))
    d2 = d1 - sigma*np.sqrt(tau)
    return x*norm.cdf(d1) - K*np.exp(-r*tau)*norm.cdf(d2)

# Merton call price with lognormal jumps
def merton_call(x, K, r, sigma, lam, muJ, sigJ, tau, n_max=40):
    price = 0.0
    
    # jump compensator
    k = np.exp(muJ + 0.5*sigJ**2) - 1
    
    for n in range(n_max):
        # Effective spot after n jumps
        x_n = x * np.exp(n*muJ + 0.5*n*sigJ**2 - lam*k*tau)
        
        # Poisson weight
        w_n = math.exp(-lam*tau) * (lam*tau)**n / math.factorial(n)
        
        price += w_n * bs_call(x_n, K, r, sigma, tau)
    
    return price

# Example
S0 = 100
K = 100
r = 0.02
sigma = 0.20
lam = 0.9
muJ = -0.10
sigJ = 0.15
tau = 1.0

print("Merton call price:", merton_call(S0, K, r, sigma, lam, muJ, sigJ, tau))

Merton call price: 9.553486972531095


Increasing λ or making jumps more negative (trough μ_J deacreasing) lowers the call price and steepens the implied-volatility skew.
When λ tends to 0 the price converges smoothly to Black–Scholes pricing.

# 5. Quadratic-Risk Hedging – Practical Approximation

Exact Merton delta involves:

$$\frac{\partial}{\partial x} F(t,x) = \sum_{n=0}^{\infty} w_n \frac{\partial}{\partial x} C_{BS}(x_n)$$

with $x_n = x e^{n\mu_J + \frac{1}{2}n\sigma_J^2 - \lambda k(T-t)}$.

Rather than computing an infinite sum at every timestep, we use the standard effective-volatility hedge:

$$\sigma_{eff} = \sqrt{\sigma^2 + \lambda \mathbb{E}[Y^2]}, \quad Y \sim N(\mu_J, \sigma_J^2)$$

i.e.

$$\mathbb{E}[Y^2] = \sigma_J^2 + \mu_J^2.$$

Then

$$\Delta_{jump}(t,x) = \frac{\partial}{\partial x} BS(x, K, r, \sigma_{eff}, \tau).$$

This hedge is not perfect (market incomplete), but reduces risk compared with pure BS delta.


In [ ]:
def bs_delta(x, K, r, sigma, tau):
    d1 = (np.log(x/K) + (r + 0.5*sigma**2)*tau)/(sigma*np.sqrt(tau))
    return norm.cdf(d1)

def jump_delta(x, K, r, sigma, lam, muJ, sigJ, tau):
    # effective vol
    var_jump = muJ**2 + sigJ**2
    sigma_eff = np.sqrt(sigma**2 + lam * var_jump)
    return bs_delta(x, K, r, sigma_eff, tau)

# 6. Hedging Error Simulation

We compare, on simulated jump–diffusion paths:
- a Black–Scholes delta hedge (ignoring jumps),
- a jump-aware proxy hedge (using an “effective” volatility inflated by jump risk),

and study the distribution of the hedging error at maturity.

Expected outcome :

- BS hedge RMSE > Jump-aware hedge RMSE
- both strictly > 0 (market incomplete)

In [ ]:
def hedge_path(S0, K, r, sigma, lam, muJ, sigJ, T, steps):
    dt = T/steps
    
    X = S0
    V_bs = 0.0
    V_jump = 0.0
    
    # initial deltas
    tau = T
    delta_bs = bs_delta(X, K, r, sigma, tau)
    delta_jump = jump_delta(X, K, r, sigma, lam, muJ, sigJ, tau)
    
    cash_bs = 0.0
    cash_jump = 0.0
    
    for t in range(steps):
        tau = T - t*dt
        
        # evolve underlying
        dW = np.sqrt(dt)*np.random.randn()
        N_jump = np.random.poisson(lam*dt)
        jump = np.exp(muJ + sigJ*np.random.randn()) - 1 if N_jump>0 else 0.0
        
        X_new = X * np.exp((r-0.5*sigma**2)*dt + sigma*dW) * (1+jump)
        
        # portfolio updates
        cash_bs *= np.exp(r*dt)
        cash_jump *= np.exp(r*dt)
        
        cash_bs -= delta_bs * (X_new - X)
        cash_jump -= delta_jump * (X_new - X)
        
        # update hedge ratios
        delta_bs = bs_delta(X_new, K, r, sigma, tau)
        delta_jump = jump_delta(X_new, K, r, sigma, lam, muJ, sigJ, tau)
        
        X = X_new
    
    payoff = max(X - K, 0)
    V_bs = cash_bs + delta_bs * X
    V_jump = cash_jump + delta_jump * X
    
    return payoff - V_bs, payoff - V_jump

# Monte-Carlo
N = 5000
errors_bs = []
errors_jump = []

for _ in range(N):
    e_bs, e_j = hedge_path(S0, K, r, sigma, lam, muJ, sigJ, tau, 200)
    errors_bs.append(e_bs)
    errors_jump.append(e_j)

print("RMSE – BS hedge:", np.sqrt(np.mean(np.array(errors_bs)**2)))
print("RMSE – Jump-aware hedge:", np.sqrt(np.mean(np.array(errors_jump)**2)))

RMSE – BS hedge: 55.25679456880206
RMSE – Jump-aware hedge: 54.8503102813989


With RMSE ≈ 55.26 for the Black–Scholes hedge and RMSE ≈ 54.85 for the jump-aware hedge, the jump-adjusted strategy performs slightly better, confirming that accounting for jumps reduces (but does not eliminate) hedging risk in an incomplete market.